In [1]:
from pystac_client import Client
import eofetch
import os
import zipfile
import track_datetime

#### Navigate the catalog and filter

In [2]:
catalog_url = 'https://eocat.esa.int/eo-catalogue'
catalog = Client.open(catalog_url)

In [3]:
# Check out https://earthcarehandbook.earth.esa.int/catalogue/index for product descriptions
# prod_type='CPR_CLD_2A' #Cloud profiling radar
prod_type='ATL_ICE_2A'

search = catalog.search(
    #collections = ['EarthCAREL1Validated'], # uncommment for searching L1 data
    collections = ['EarthCAREL2Validated'], # uncomment for searching L2 data 
    filter = f"productType='{prod_type}'", # Filter by product type, there are more options for filtering here too! 
    datetime = ['2025-04-01T00:00:00.000Z', '2025-04-30T00:00:00.000Z'], # filter by temporal extent 
    method = 'GET',
    bbox = [8, 76, 33, 81], #or [9, 43, 23, 47], # bounding box is defined by the bottom left corner (longmin latmin) and the top right corner coordinates (longmax latmax) 

    max_items=100  # Adjust as needed, if you don't add max_items over 100000s of products though this could take really long 
)

items = list(search.items())

id_lst = []
h_lst = []

In [4]:
for item in items:
    if prod_type in item.id: id_lst.append(item.assets["enclosure"].href) 

In [ ]:
# Need to set your username and password for Earth login / OADS
os.environ["OADS_USERNAME"] = "******"
os.environ["OADS_PASSWORD"] = "#*****"

In [10]:
# download the data as .zip files
eofetch.download(id_lst)

The following will unzip the downloaded files and place them into a directory strucuture which should be easy to use.
A `track_datetime` object is created and saves the dates and times of tracks of the region of interest. This can be saved to be reloaded into your own analysis.

In [ ]:
loc='svalbard' #setting this so that we can name an object
               #which contains the dates / times of passes over a region. 
               
#################################################
### Uncomment below to read in pickled object ###
#################################################

pklpath=f'/work/bb1086/b383354/earth-care-data/track_datetimes/{loc}_tracks_times.pkl'
if os.path.isfile(pklpath):
    dates_obj=track_datetime.read_from_pickle(pklpath) 
else:
    dates_obj=track_datetime.track_dates()


for file in id_lst:
    fi=file.rsplit('/',1)[1]
    a=fi.rsplit('_',3)[1]
    yy=a[0:4]
    mm=a[4:6]
    dd=a[6:8]
    hr=a[9:11]
    mn=a[11:13]

    
    path=f'/work/bb1086/b383354/earth-care-data/{yy}/{mm}/{dd}'  #This is my path on work, you can set to your own
                                        
    if not os.path.isdir(path):
        os.makedirs(path, exist_ok=True)
    
        try:
            with zipfile.ZipFile(fi, 'r') as zip_ref:
                zip_ref.extract(f"{fi.rsplit('.',1)[0]}.h5", path=path) #extract the files work directory structure
                dates_obj.add_time(date=f'{yy}-{mm}-{dd}',time=f'T{hr}:{mn}:00')
                
        except:
            print(f'could not unzip file {fi}')
            
    os.remove(f'./{fi}') #remove the downloaded file


## maybe should save pickles with instrument name too?

if not os.path.isdir(f'{path}/../../../track_datetimes'):
    os.makedirs(f'{path}/../../../track_datetimes', exist_ok=True)
dates_obj.save_pickle(f'{path}/../../../track_datetimes/{loc}_track_datetimes.pkl')

